# 데이터 작업하기

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

- 파이토치에는 데이터 작업을 위한 기본 요소 두 가지인 utils.data.DataLoader와 utils.data.Dataset이 있습니다. Dataset은 샘플과 정답(Label)을 저장하고, DataLoader는 Dataset을 순회 가능한 객체(iterable)로 감쌉니다.

torchvision은 이미지 처리(비전 데이터)에 대한 도메인 특화 라이브러리입니다. 모든 TorchVision Dataset은 샘플과 정답을 각각 변경하기 위한 transform과 target_transform의 두 인자를 포함합니다.

In [2]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),)

# 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),)

Dataset을 DataLoader의 인자로 전달합니다. 이는 데이터셋을 순회 가능한 객체(iterable)로 감싸고, 자동화된 배치(batch), 샘플링(sampling), 섞기(shuffle) 및 다중 프로세스로 데이터 불러오기(multiprocess data loading)을 지원합니다. 여기서는 배치 크기(batch size)를 64로 정의합니다. 즉, 데이터로더 객체의 각 요소는 64개의 특징(feature)과 정답(label)을 묶음(batch)로 반환합니다.

In [3]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# 모델 만들기
PyTorch에서 신경망 모델은 nn.Module을 상속받는 클래스를 생성하여 정의합니다. __init__ 함수에서 신경망의 계층(layer)들을 정의하고 forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정합니다. 가능한 경우 GPU로 신경망을 이동시켜 연산을 가속(accelerate)합니다. 

In [4]:
# 학습에 사용할 CPU나 GPU 장치의 정보를 가져옵니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [5]:
# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# 모델 매개변수 최적화하기
모델을 학습하려면 손실 함수(loss function)와 옵티마이저(optimizer)가 필요합니다.

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

각 학습 단계(training loop)에서 모델은 (배치로 제공되는) 학습 데이터셋에 대한 예측을 수행하고, 예측 오류를 역전파하여 모델의 매개변수를 조정합니다.

In [7]:
len(train_dataloader) * 64

60032

In [8]:
len(train_dataloader.dataset.data)

60000

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

모델의 학습 결과를 확인하기 위해 테스트 데이터셋에 대한 코드도 작성합니다.

In [14]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습 단계는 여러 번의 반복 단계 (Epochs)를 거쳐서 수행됩니다. 각 에폭에서 모델은 더 나은 예측을 하기 위해 매개변수를 학습합니다. 각 에폭마다 모델의 정확도와 손실을 출력합니다. 에폭마다 정확도와 손실의 증감폭을 확인할 것입니다.

In [15]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306738  [    0/60000]
loss: 2.298526  [ 6400/60000]
loss: 2.276097  [12800/60000]
loss: 2.272758  [19200/60000]
loss: 2.250129  [25600/60000]
loss: 2.218248  [32000/60000]
loss: 2.229562  [38400/60000]
loss: 2.192589  [44800/60000]
loss: 2.195400  [51200/60000]
loss: 2.156508  [57600/60000]
Test Error: 
 Accuracy: 44.5%, Avg loss: 2.153089 

Epoch 2
-------------------------------
loss: 2.164933  [    0/60000]
loss: 2.162782  [ 6400/60000]
loss: 2.096381  [12800/60000]
loss: 2.114935  [19200/60000]
loss: 2.060219  [25600/60000]
loss: 1.997635  [32000/60000]
loss: 2.028604  [38400/60000]
loss: 1.941573  [44800/60000]
loss: 1.956593  [51200/60000]
loss: 1.877472  [57600/60000]
Test Error: 
 Accuracy: 54.2%, Avg loss: 1.876428 

Epoch 3
-------------------------------
loss: 1.908940  [    0/60000]
loss: 1.891292  [ 6400/60000]
loss: 1.758031  [12800/60000]
loss: 1.803810  [19200/60000]
loss: 1.694401  [25600/60000]
loss: 1.640995  [32000/600

# 모델 저장하기
모델을 저장하는 일반적인 방법은 (모델의 매개변수들을 포함하여) 내부 상태 사전(internel state dictionary)을 직렬화(serialize)하는 것입니다.

In [16]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# 모델 불러오기
모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함됩니다.

In [17]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

이제 이 모델을 사용해서 예측을 할 수 있습니다.

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
